In [1]:
from langchain_community.document_loaders import PyPDFLoader
import os
import getpass
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
openai_api_key = os.getenv('OPENAI_API_KEY', 'YourAPIKeyIfNotSet')

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')



In [9]:
loader = PyMuPDFLoader("~/Dropbox/LLM-agriculture/PDFs/UV-B照射によるいちごうどんこ病等の防除体系の確立.pdf")
pages = loader.load_and_split()


ImportError: `PyMuPDF` package not found, please install it with `pip install pymupdf`

In [5]:


def make_embedder(model_type, save_path):
    if model_type == "sup-simcse-ja-base":
        model_name = "cl-nagoya/sup-simcse-ja-base"
        model_kwargs = {'device': 'cpu'}
        encode_kwargs = {'normalize_embeddings': False}
        embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
            )
    else:  # Assuming OpenAIEmbeddings
        embedding_model = OpenAIEmbeddings()
    
    fs = LocalFileStore(save_path)
    cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model, fs)
    return cached_embedder

def save_embeddings_and_ids_with_faiss(all_documents, cached_embedder, save_path):
    db = FAISS.from_documents(all_documents, cached_embedder)
    db.save_local(save_path)

model_type = "sup-simcse-ja-base"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

cached_embedder = make_embedder(model_type, save_path)
save_embeddings_and_ids_with_faiss(pages, cached_embedder, save_path)

In [6]:
model_type = "sup-simcse-ja-base"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

model_name = "cl-nagoya/sup-simcse-ja-base"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
            )
fs = LocalFileStore(save_path)
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model, fs)
db = FAISS.load_local(save_path, cached_embedder)

In [7]:

llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())

In [9]:
query = "うどんこ病の福島県での発生状況は？"
qa.invoke(query)

{'query': 'うどんこ病の福島県での発生状況は？',
 'result': ' 県内全域で普通にみられ、イチゴ品種により感受性に差があり、「さちのか」や「ふくはる香」は感染しやすい。'}

In [10]:
query = "前年から鉢を使い回すときの注意点は？"
qa.invoke(query)

{'query': '前年から鉢を使い回すときの注意点は？',
 'result': ' 前作使用した育苗土壌やポット等の資材は、消毒するか一斉に更新することが重要です。また、土壌消毒を行い次作に病原菌を持ち越さないようにすることも大切です。'}

In [11]:
query = "炭疽病の発生時期はいつ？"
qa.invoke(query)

{'query': '炭疽病の発生時期はいつ？',
 'result': ' 高温時期に発生しやすい。気温28℃の比較的高温条件で多発する。定植後秋口にすぐ症状が見られるものは、潜在感染した親株から採苗～育苗期に感染したと考えられる。'}